# Importing Lib and docs

In [29]:
# Pandas and Numpy
import numpy as np
import pandas as pd

# Charting and plotting
from matplotlib import pyplot as plt
import seaborn as sns
from pylab import rcParams
rcParams['figure.figsize'] = 15,8

# Ignore Warnings
from warnings import filterwarnings
filterwarnings("ignore")

In [30]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Installing Medipipe

In [31]:
# installing MediaPipe
!pip install -q mediapipe==0.10.0

In [32]:
hand_model_path = '/content/drive/MyDrive/Mtech_Project/Code/hand_landmarker.task'
face_model_path = '/content/drive/MyDrive/Mtech_Project/Code/face_landmarker.task'
pose_model_path = '/content/drive/MyDrive/Mtech_Project/Code/pose_landmarker.task'

In [33]:
# Import the mediapipe modules and Hand and Face Landmarker API 
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

In [34]:
# visualization tools
import cv2
from google.colab.patches import cv2_imshow

## Create MP Objects

In [35]:
mp_holistic = mp.solutions.holistic

In [36]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the video mode:
options_hand = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path = hand_model_path),
    running_mode=VisionRunningMode.VIDEO)

# Create a pose landmarker instance with the video mode:
options_pose = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=pose_model_path),
    running_mode=VisionRunningMode.VIDEO)

# Create a face landmarker instance with the video mode:
options_face = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=face_model_path),
    running_mode=VisionRunningMode.VIDEO)

In [37]:
detector_face = vision.FaceLandmarker.create_from_options(options_face)
detector_hand = vision.HandLandmarker.create_from_options(options_hand)
detector_pose = vision.PoseLandmarker.create_from_options(options_pose)

# Load Image Locations

In [38]:
holistic = mp_holistic.Holistic(
    static_image_mode=False, 
    model_complexity=1, 
    smooth_landmarks=True, 
    enable_segmentation=False, 
    refine_face_landmarks=True, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
    )

In [39]:
df_img_path = pd.read_csv('/content/drive/MyDrive/Mtech_Project/Dataset_Video/file_location.csv')

In [40]:
df_img_path.head()

,Expression,File_Name,Value
0,Ethnicity,MVI_8950.MP4,/content/drive/MyDrive/Mtech_Project/Dataset_V...
1,Ethnicity,MVI_8688.MP4,/content/drive/MyDrive/Mtech_Project/Dataset_V...
2,Ethnicity,MVI_4821.MOV,/content/drive/MyDrive/Mtech_Project/Dataset_V...
3,Ethnicity,MVI_4823.MOV,/content/drive/MyDrive/Mtech_Project/Dataset_V...
4,Ethnicity,MVI_4822.MOV,/content/drive/MyDrive/Mtech_Project/Dataset_V...


In [41]:
str_t = df_img_path['File_Name'][1]

In [42]:
str_t[:-4]

'MVI_8688'

# Creating Dataframe structure

In [43]:
li_headers = []
hand_points = mp_holistic.HandLandmark
for point in hand_points:
  x = str(point)[13:]
  li_headers.append('LEFT_' + x + '_x')
  li_headers.append('LEFT_' + x + '_y')
  li_headers.append('LEFT_' + x + '_z')
  li_headers.append('LEFT_' + x + '_vis')
for point in hand_points:
  x = str(point)[13:]
  li_headers.append('RIGHT_' + x + '_x')
  li_headers.append('RIGHT_' + x + '_y')
  li_headers.append('RIGHT_' + x + '_z')
  li_headers.append('RIGHT_' + x + '_vis')
pose_points = mp_holistic.PoseLandmark
for point in pose_points:
  x = str(point)[13:]
  li_headers.append('POSE_' + x + '_x')
  li_headers.append('POSE_' + x + '_y')
  li_headers.append('POSE_' + x + '_z')
  li_headers.append('POSE_' + x + '_vis')
li_headers.insert(0,'POSE')
li_headers.insert(1,'IMG')
li_headers.insert(2,'FRAME')
li_headers.insert(3,'TS')

In [44]:
df_landmark = pd.DataFrame(columns=li_headers)

In [45]:
df_landmark

,POSE,IMG,FRAME,TS,LEFT_WRIST_x,LEFT_WRIST_y,LEFT_WRIST_z,LEFT_WRIST_vis,LEFT_THUMB_CMC_x,LEFT_THUMB_CMC_y,...,POSE_RIGHT_HEEL_z,POSE_RIGHT_HEEL_vis,POSE_LEFT_FOOT_INDEX_x,POSE_LEFT_FOOT_INDEX_y,POSE_LEFT_FOOT_INDEX_z,POSE_LEFT_FOOT_INDEX_vis,POSE_RIGHT_FOOT_INDEX_x,POSE_RIGHT_FOOT_INDEX_y,POSE_RIGHT_FOOT_INDEX_z,POSE_RIGHT_FOOT_INDEX_vis


# Conversion of Videos into pose

In [46]:
for idx, loc in enumerate(df_img_path['Value']):
  cap = cv2.VideoCapture(loc)
  frame_no = 0
  while cap.isOpened(): 
      success, image = cap.read()
      if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        break

      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = holistic.process(image)

      frame_li = []
      frame_li.append(df_img_path['Expression'][idx])
      str_tmp = df_img_path['File_Name'][idx]
      frame_li.append(str_tmp[:-4])
      frame_li.append(frame_no)
      frame_li.append(str(cap.get(cv2.CAP_PROP_POS_MSEC)))
      
      # Load the landmarks to a dataframe

      hand_points = mp_holistic.HandLandmark
      if results.left_hand_landmarks:
        left_landmarks = results.left_hand_landmarks.landmark
        for i,j in zip(hand_points, left_landmarks):
          frame_li.append(j.x)
          frame_li.append(j.y)
          frame_li.append(j.z)
          frame_li.append(j.visibility)
      else:
        for i in hand_points:
          frame_li.append(0)
          frame_li.append(0)
          frame_li.append(0)
          frame_li.append(0)
      
      if results.right_hand_landmarks :
        right_landmarks = results.right_hand_landmarks.landmark
        for i,j in zip(hand_points, right_landmarks):
          frame_li.append(j.x)
          frame_li.append(j.y)
          frame_li.append(j.z)
          frame_li.append(j.visibility)
      else:
        for i in hand_points:
          frame_li.append(0)
          frame_li.append(0)
          frame_li.append(0)
          frame_li.append(0)

      pose_points = mp_holistic.PoseLandmark
      if results.pose_landmarks:
        pose_landmarks = results.pose_landmarks.landmark
        for i,j in zip(pose_points, pose_landmarks):
          frame_li.append(j.x)
          frame_li.append(j.y)
          frame_li.append(j.z)
          frame_li.append(j.visibility)
      else:
        for i in pose_points:
          frame_li.append(0)
          frame_li.append(0)
          frame_li.append(0)
          frame_li.append(0)
      #list_landmark.append(results.left_hand_landmarks) 
      #list_landmark.append(results.right_hand_landmarks) 
      #list_landmark.append(results.pose_landmarks) 
      frame_no += 1
      df_landmark.loc[len(df_landmark)] = frame_li
      print(df_landmark.loc[-1:])

  cap.release()
  cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.
14114                  0.021124                 0.475514   
14115                  0.019070                 0.476361   
14116                  0.017227                 0.478155   

       POSE_RIGHT_FOOT_INDEX_y  POSE_RIGHT_FOOT_INDEX_z  \
0                     1.183646                 0.255756   
1                     1.212572                 0.164810   
2                     1.217781                 0.164584   
3                     1.215270                 0.173151   
4                     1.213317                 0.183877   
...                        ...                      ...   
14112                 1.308169                 0.267573   
14113                 1.315030                 0.256739   
14114                 1.317557                 0.275084   
14115                 1.320941                 0.274665   
14116                 1.324736                 0.248380   

       POSE_RIGHT_FOOT_INDEX_vis  
0                       

In [47]:
df_landmark.tail(10)

,POSE,IMG,FRAME,TS,LEFT_WRIST_x,LEFT_WRIST_y,LEFT_WRIST_z,LEFT_WRIST_vis,LEFT_THUMB_CMC_x,LEFT_THUMB_CMC_y,...,POSE_RIGHT_HEEL_z,POSE_RIGHT_HEEL_vis,POSE_LEFT_FOOT_INDEX_x,POSE_LEFT_FOOT_INDEX_y,POSE_LEFT_FOOT_INDEX_z,POSE_LEFT_FOOT_INDEX_vis,POSE_RIGHT_FOOT_INDEX_x,POSE_RIGHT_FOOT_INDEX_y,POSE_RIGHT_FOOT_INDEX_z,POSE_RIGHT_FOOT_INDEX_vis
14161,Spring,MVI_5428,73,2920.0000000000005,0.571369,0.768938,1.546709e-07,0.0,0.555746,0.781674,...,0.290012,0.041206,0.531689,1.288041,0.236489,0.020246,0.474174,1.292244,0.129827,0.019925
14162,Spring,MVI_5428,74,2960.0000000000005,0.568457,0.783313,1.414789e-07,0.0,0.553294,0.792546,...,0.274939,0.041742,0.532126,1.294624,0.233973,0.022162,0.479589,1.300937,0.107571,0.022048
14163,Spring,MVI_5428,75,3000.0000000000005,0.565345,0.796777,1.340551e-07,0.0,0.550519,0.801207,...,0.254901,0.042655,0.532295,1.308174,0.229668,0.025127,0.478635,1.320035,0.083283,0.025101
14164,Spring,MVI_5428,76,3040.0,0.562748,0.802131,1.124784e-07,0.0,0.548157,0.804883,...,0.241163,0.044641,0.533927,1.306869,0.225263,0.029078,0.476259,1.316331,0.077150,0.029293
14165,Spring,MVI_5428,77,3080.0,0.560867,0.805059,9.929091e-08,0.0,0.547395,0.806760,...,0.232370,0.046103,0.535344,1.306505,0.221351,0.032378,0.475667,1.314111,0.072140,0.032593
14166,Spring,MVI_5428,78,3120.0,0.562648,0.803918,1.022545e-07,0.0,0.549870,0.805031,...,0.237061,0.047646,0.536271,1.304733,0.226563,0.036059,0.475485,1.312732,0.075557,0.036403
14167,Spring,MVI_5428,79,3160.0,0.564917,0.802167,1.072509e-07,0.0,0.551870,0.802539,...,0.239860,0.050385,0.537461,1.303285,0.231114,0.040519,0.475681,1.311824,0.086604,0.041055
14168,Spring,MVI_5428,80,3200.0,0.565264,0.801494,1.089753e-07,0.0,0.552051,0.801875,...,0.267998,0.052262,0.537566,1.300509,0.230706,0.043489,0.476484,1.309763,0.105343,0.044218
14169,Spring,MVI_5428,81,3240.0,0.566169,0.800981,1.045337e-07,0.0,0.552468,0.802262,...,0.284737,0.054380,0.538036,1.297260,0.248271,0.047030,0.476341,1.307344,0.122918,0.047473
14170,Spring,MVI_5428,82,3280.0000000000005,0.566563,0.801300,1.166212e-07,0.0,0.552436,0.802575,...,0.306758,0.055673,0.538038,1.295424,0.253777,0.049992,0.477327,1.305604,0.145021,0.049777


# Saving Dataframe to CSV

In [48]:
filepath = '/content/drive/MyDrive/Mtech_Project/Dataset_Video/landmark_1.csv'
df_landmark.to_csv(filepath)